In [2]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import os
import numpy as np
import pandas as pd
import re
import time
import pdb
import random
from multiprocessing.dummy import Pool as ThreadPool

In [3]:
home='http://sousuo.gov.cn/column/31421/'

In [4]:
T=list()
for i in range(0,215):
    I=str(i)
    T.append(home+I+'.htm')

In [4]:
#T

In [5]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36"}

In [6]:
s = requests.session()   #会话保持
s.keep_alive = False
s.header = headers   #设置请求头

In [7]:
url=[]
for k in range(len(T)):
    response = s.get(T[k], headers = headers,timeout = 5)   #使用get方法获取网页，得到一个response对象
    soup = BeautifulSoup(response.content,'html.parser')    #.content显示bytes型网页的源代码，'html.parser'为解析器
    Link=soup.find_all("div",class_="list list_1 list_2")[0].find_all("a")   #find_all()返回列表，取文章标题   
    for link in Link:
        url.append(link.get('href'))   #取原文链接，添加到url列表中

In [8]:
len(url)

4300

In [9]:
bcurl=pd.DataFrame(url)   #创建数据框
bcurl.to_excel(r'C:\\Users\\X1\\Desktop\\data.xlsx','sheet1',na_rep='nana',header=["url"])

In [10]:
bcurl=pd.read_excel(r'C:\\Users\\X1\\Desktop\\data.xlsx')

In [11]:
bcurl["url"]

0       http://www.gov.cn/zhengce/2021-11/26/content_5...
1       http://www.gov.cn/xinwen/2021-11/26/content_56...
2       http://www.gov.cn/zhengce/content/2021-11/26/c...
3       http://www.gov.cn/premier/2021-11/26/content_5...
4       http://www.gov.cn/premier/2021-11/26/content_5...
                              ...                        
4295    http://www.gov.cn/xinwen/2021-01/05/content_55...
4296    http://www.gov.cn/xinwen/2021-01/05/content_55...
4297    http://www.gov.cn/xinwen/2021-01/05/content_55...
4298    http://www.gov.cn/xinwen/2021-01/05/content_55...
4299    http://www.gov.cn/guowuyuan/2021-01/05/content...
Name: url, Length: 4300, dtype: object

In [ ]:
response = s.get(bcurl["url"][6], headers = headers,timeout = 5)
soup = BeautifulSoup(response.content,'html.parser')

In [ ]:
#soup

In [ ]:
for p in soup.select('p'):   #输出的列表含有多个包含标签p的元素
        t = p.get_text()   #get_text()清空所有html标签元素，返回干净的文字
        print(t)

In [ ]:
#words

In [ ]:
response.encoding = 'utf-8'
hhh=response.text   #返回的是一个 unicode 型的文本数据

In [ ]:
#hhh

In [ ]:
tit = re.findall(r'<td><b>标\u3000\u3000题：</b></td>\r\n                <td colspan="3">(.*?)</td>\r\n',hhh,re.S)[0]
#返回hhh中所有与pattern匹配的全部字符串,返回形式为数组。

In [ ]:
tit

In [ ]:
date=re.findall(r'<td><b>发布日期：</b></td>\r\n                <td>(.*?)</td>\r\n',hhh,re.S)[0]

In [ ]:
date

In [ ]:
#zt=re.findall(r'<b>主题分类：</b></td>\r\n                <td width="360">(.*?)</td>\r\n',hhh,re.S)[0]

In [ ]:
#zt

In [12]:
def content(url): 
    time.sleep(random.uniform(0.1, 0.2))   #暂停执行0.1或0.2秒
    try:
        response = s.get(url, headers = headers,timeout = 5)
        soup = BeautifulSoup(response.content,'html.parser')
        response.encoding = 'utf-8'
        hhh=response.text   
        text=soup.find_all("div",class_="wrap")[0].find_all({"p"})
        word=[]
        for i in text:
            word.append("".join(i.strings))
        words=''.join(word)
        title = re.findall(r'<td><b>标\u3000\u3000题：</b></td>\r\n                <td colspan="3">(.*?)</td>\r\n ',hhh,re.S)[0]
        date=re.findall(r'<td><b>发布日期：</b></td>\r\n                <td>(.*?)</td>\r\n',hhh,re.S)[0]
        #zt=re.findall(r'<b>主题分类：</b></td>\r\n                <td width="360">(.*?)</td>\r\n',hhh,re.S)[0]
        return([words,title,date,url])    
    except:
        pass    

In [13]:
pool = ThreadPool(3)          #设置多少个线程 -- 不要设置太大，不然短时间内访问次数过多，会被反爬
web_data = bcurl['url'][0:4299].tolist()   #转换成列表
len(web_data)

4299

In [14]:
### 正式爬取——并行
#pool = ThreadPool(3)          #设置多少个线程 -- 不要设置太大，不然短时间内访问次数过多，会被反爬
#web_data = zcurl['zcurl'][0:1225].tolist()   
#这一步可以分开爬，因为全部网址依然很多，例如爬取1万-2万的
#web_data = agency_urls['网址'][9000:9110].tolist()

# all data 存储数据
all_data = []
# 分成不同个batch进行爬取
irt = np.arange(0,len(web_data),100); irt = np.append(irt,len(web_data))
all_time = 0
for i in range(0,len(irt)-1):
    # 取第一个batch
    t0 = time.time()
    temp_data = web_data[irt[i]:irt[i+1]]
    # 并行爬取，返回一个list
    temp_result = list(map(content, temp_data))   #将content函数依次作用在temp_data上
    # 合并list，记录数据
    all_data = all_data + temp_result
    t1 = time.time()
    all_time = all_time + t1 - t0
    print('batch' + str(irt[i+1]) + '用时:', t1 - t0 )
    # print(irt[i+1])
print('总用时:', all_time)

batch100用时: 32.11039113998413
batch200用时: 38.846240758895874
batch300用时: 38.91755127906799
batch400用时: 39.29150176048279
batch500用时: 38.931482791900635
batch600用时: 38.24862003326416
batch700用时: 39.66687226295471
batch800用时: 36.8598051071167
batch900用时: 40.356446743011475
batch1000用时: 38.86558699607849
batch1100用时: 45.873562812805176
batch1200用时: 39.49180626869202
batch1300用时: 39.208895444869995
batch1400用时: 40.91545224189758
batch1500用时: 42.50090765953064
batch1600用时: 40.48216891288757
batch1700用时: 38.91006779670715
batch1800用时: 37.87273073196411
batch1900用时: 37.06164908409119
batch2000用时: 53.56438875198364
batch2100用时: 36.14893841743469
batch2200用时: 34.398388385772705
batch2300用时: 35.27322173118591
batch2400用时: 36.37076783180237
batch2500用时: 33.723841428756714
batch2600用时: 34.06964325904846
batch2700用时: 34.56345248222351
batch2800用时: 36.22802472114563
batch2900用时: 34.598331689834595
batch3000用时: 35.50547432899475
batch3100用时: 36.49254035949707
batch3200用时: 35.73389506340027
batch3300用

In [15]:
all_data1=[]
for i in range(len(all_data)):
    if all_data[i]!=None:
        all_data1.append(all_data[i])

In [16]:
all_data

[None,
 None,
 ['国务院关于支持北京城市副中心高质量发展的意见国发〔2021〕15号\n各省、自治区、直辖市人民政府，国务院各部委、各直属机构：规划建设北京城市副中心（以下简称城市副中心），与河北雄安新区形成北京新的两翼，是以习近平同志为核心的党中央作出的重大决策部署，是千年大计、国家大事。为深入贯彻落实党中央、国务院决策部署，进一步支持城市副中心高质量发展，现提出以下意见。一、总体要求（一）指导思想。以习近平新时代中国特色社会主义思想为指导，深入贯彻党的十九大和十九届二中、三中、四中、五中全会精神，按照党中央、国务院决策部署，坚持稳中求进工作总基调，立足新发展阶段，完整、准确、全面贯彻新发展理念，构建新发展格局，统筹发展和安全，围绕创造“城市副中心质量”，坚持创新引领，提高治理水平，推动绿色发展，深化改革开放，提升和谐宜居品质，有序承接北京非首都功能疏解，努力推动实现更高质量、更有效率、更加公平、更可持续、更为安全的发展，引领带动周边地区一体化发展，打造京津冀协同发展的高质量样板和国家绿色发展示范区，为建设和谐、宜居、美丽的大国首都作出贡献。（二）基本原则。——承接疏解、错位发展。牢牢抓住疏解北京非首都功能这个“牛鼻子”，有序承接符合城市副中心发展定位的功能疏解和人口转移，提升对首都功能的服务保障能力，实现以副辅主、主副共兴，与河北雄安新区各有分工、互为促进，有效解决北京“大城市病”问题。——改革创新、试点示范。坚持创新驱动发展，根据发展定位和实际需要，赋予更大发展自主权，提升科技创新、行政办公、商务服务、文化和旅游等功能，推动政策措施和试点示范项目先行先试，形成可复制、可推广的经验。——协同联动、一体发展。按照统一规划、统一政策、统一标准、统一管控（以下称四统一）要求，积极推进城市副中心、通州区与河北省三河市、大厂回族自治县、香河县（以下称北三县）一体化高质量发展，探索逐步实现共同富裕的新路径，为推进京津冀协同发展作出示范。（三）主要目标。到2025年，城市副中心绿色城市、森林城市、海绵城市、智慧城市、人文城市、宜居城市功能基本形成。北京市级党政机关和市属行政事业单位搬迁基本完成，承接北京非首都功能疏解和人口转移取得显著成效，城市管理和社会治理水平明显提升，现代产业体系初步形成，“城市副中心质量”体系初步构建。通州区与北三县一体化高质量发展制度体

In [17]:
df= pd.DataFrame(pd.DataFrame(all_data1))
df.columns = ['正文','标题','日期',"url"]

In [18]:
df

,正文,标题,日期,url
0,国务院关于支持北京城市副中心高质量发展的意见国发〔2021〕15号\n各省、自治区、直辖市人...,国务院关于支持北京城市副中心高质量发展的意见,2021年11月26日,http://www.gov.cn/zhengce/content/2021-11/26/c...
1,国务院关于开展营商环境创新试点工作的意见国发〔2021〕24号\n各省、自治区、直辖市人民政...,国务院关于开展营商环境创新试点工作的意见,2021年11月25日,http://www.gov.cn/zhengce/content/2021-11/25/c...
2,国务院关于“十四五”对外贸易高质量发展规划的批复国函〔2021〕112号\n各省、自治区、直...,国务院关于“十四五”对外贸易高质量发展规划的批复,2021年11月23日,http://www.gov.cn/zhengce/content/2021-11/23/c...
3,国务院办公厅关于进一步加大对中小企业纾困帮扶力度的通知国办发〔2021〕45号\n各省、自治...,国务院办公厅关于进一步加大对中小企业纾困帮扶力度的通知,2021年11月22日,http://www.gov.cn/zhengce/content/2021-11/22/c...
4,国务院办公厅关于健全重特大疾病医疗保险和救助制度的意见国办发〔2021〕42号\n各省、自治...,国务院办公厅关于健全重特大疾病医疗保险和救助制度的意见,2021年11月19日,http://www.gov.cn/zhengce/content/2021-11/19/c...
...,...,...,...,...
111,国务院办公厅关于进一步做好困难群众\n基本生活保障有关工作的通知\n国办发明电〔2021〕2...,国务院办公厅关于进一步做好困难群众基本生活保障有关工作的通知,2021年01月21日,http://www.gov.cn/zhengce/content/2021-01/21/c...
112,国务院应对新型冠状病毒感染肺炎疫情联防联控机制关于进一步做好当前新冠肺炎疫情防控工作的通知国...,国务院应对新型冠状病毒感染肺炎疫情联防联控机制关于进一步做好当前新冠肺炎疫情防控工作的通知,2021年01月20日,http://www.gov.cn/zhengce/content/2021-01/20/c...
113,中华人民共和国国务院令第734号\n《企业名称登记管理规定》已经2020年12月14日国务院...,企业名称登记管理规定,2021年01月19日,http://www.gov.cn/zhengce/content/2021-01/19/c...
114,国务院办公厅转发国家发展改革委等部门关于清理规范城镇供水供电供气供暖行业收费促进行业高质量发...,国务院办公厅转发国家发展改革委等部门关于清理规范城镇供水供电供气供暖行业收费促进行业高质量发...,2021年01月06日,http://www.gov.cn/zhengce/content/2021-01/06/c...


In [19]:
df.to_excel(r'C:\Users\X1\Desktop\国务院.xlsx')